In [105]:
from transformers import *
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [106]:
df = pd.read_csv('data/labeled_data.csv')
data = df[['class', 'tweet']]
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [107]:
tweets = data['tweet']
tokenized = tweets.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [101]:
padded.shape

(24783, 484)

In [102]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(24783, 484)

In [ ]:
# https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

In [108]:
model_class, tokenizer_class, pretrained_weights = (DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

NameError: name 'DistilBertModel' is not defined

***

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['class'], random_state=123)
X_train.shape, X_test.shape

((18587,), (6196,))

In [4]:
train_examples = X_train.shape[0]
test_examples = X_test.shape[0]

In [5]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [35]:
def convert_to_input_example(X, y):
    df_converted = []
    for i in range(X.shape[0]):
        input_example = InputExample(guid=None, text_a=X[i], label=y[i])
        df_converted.append(input_example)
    return df_converted

In [36]:
df_train = convert_to_input_example(X_train, y_train)
df_test = convert_to_input_example(X_test, y_test)

In [93]:
X_train_encoded = X_train.apply(lambda x: tf.constant(tokenizer.encode(x, add_special_tokens=True)))
X_test_encoded = X_test.apply(lambda x: tf.constant(tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
input_ids = torch.tensor(np.array(padded))

In [86]:
list(X_train_encoded[:3])

[<tf.Tensor: id=518785, shape=(50,), dtype=int32, numpy=
 array([  101,   137, 21718,  2858, 21132, 24959,   146,   112,   182,
          170,   175,  8517, 18855,   131,   120,  8413,   131,   120,
          120,   189,   119,  1884,   120,   144,  1197,  1545, 24458,
         5253,  8057,  1161,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])>,
 <tf.Tensor: id=518786, shape=(50,), dtype=int32, numpy=
 array([  101,  2825,   112,   189,  2484,  1165,   146,  1267,   170,
         2130,  1198,  1243,  1149,  1104,  2398,   111,  1821,  1643,
          132,   123,  1552,  1224,  1147,   107,  2520,   107,  1106,
         1330,  2564,   117, 16358,  1279,  6100,  1129, 16358,  1279,
          188,  1306,  1324,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])>,
 <tf.Tensor: id=518787, shape=(50,), dtype=int32, numpy=
 array([  101,  1109, 12174

In [9]:
X_train_encodedertConfig.from_pretrained("bert-base-cased", num_labels=3)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased', config=config)

In [45]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [46]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 108,312,579
Trainable params: 108,312,579
Non-trainable params: 0
_________________________________________________________________


In [90]:
X_train_encoded.shape

(18587,)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32

history = model.fit(
    x=X_train_encoded,
    y=y_train.values,
    epochs=EPOCHS,
#     validation_data=(X_test_encoded, y_test.values)
)

In [ ]:
# tf.nn.softmax(model.evaluate())

In [42]:
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
outputs = model(input_ids)
logits = outputs[0]
print(outputs)
print(logits)

(<tf.Tensor: id=28965, shape=(1, 3), dtype=float32, numpy=array([[-0.14614113,  0.58454984,  0.26810336]], dtype=float32)>,)
tf.Tensor([[-0.14614113  0.58454984  0.26810336]], shape=(1, 3), dtype=float32)
